### **0. Import the relevant packages**

In [ ]:
import os

# Setup projects
SIMPLE_RAG_PROJECT = "simple-rag"
os.environ["PHOENIX_PROJECT_NAME"] = SIMPLE_RAG_PROJECT

In [ ]:
import pandas as pd
import phoenix as px
import nest_asyncio
from datasets import load_dataset
from dotenv import load_dotenv
from phoenix.evals import (
    HallucinationEvaluator, OpenAIModel, QAEvaluator,
    RelevanceEvaluator, run_evals
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations
from tqdm import tqdm
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import set_global_handler, Settings
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import PointStruct

nest_asyncio.apply()  # needed for concurrent evals in notebook environments
pd.set_option("display.max_colwidth", 1000)

### **1. Launch Phoenix**
You can run Phoenix in the background to collect trace data emitted by any LlamaIndex application that has been instrumented with the OpenInferenceTraceCallbackHandler. Phoenix supports LlamaIndex's one-click observability which will automatically instrument your LlamaIndex application! You can consult our integration guide for a more detailed explanation of how to instrument your LlamaIndex application.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run the LlamaIndex application).

In [ ]:
session = px.launch_app()
set_global_handler("arize_phoenix")

In [ ]:
# We can view the above data in the UI
px.active_session().view()

### **2. Run Your Query Engine and View Your Traces in Phoenix**

We've compiled a list of commonly asked questions about Qdrant. Let's download the sample queries and take a look.

In [ ]:
## Loading the Eval dataset
qdrant_qa = load_dataset("atitaarora/qdrant_docs_qna_ragas", split="train")
qdrant_qa_question = qdrant_qa.select_columns(['question'])

In [ ]:
qdrant_qa_question['question'][:10]

### *This example uses a QdrantVectorStore and uses the previously generated collection to work fully connected with Qdrant but you can use whatever LlamaIndex application you like.*

In [ ]:
load_dotenv()

In [ ]:
COLLECTION_NAME = "qdrant_docs_arize_dense"

In [ ]:
client = QdrantClient(
    os.environ.get("QDRANT_URL"), 
    api_key=os.environ.get("QDRANT_API_KEY"),
)

In [ ]:
client.get_collections()

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.core.indices.vector_store import VectorIndexRetriever
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.indices.vector_store.base import VectorStoreIndex

Settings.embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

retriever = VectorIndexRetriever(
    index=index,
    vector_store_query_mode=VectorStoreQueryMode.DEFAULT,
    similarity_top_k=5
)

In [ ]:
query_engine = index.as_query_engine()
for query in tqdm(qdrant_qa_question['question'][:10]):
    try:
      query_engine.query(query)
    except Exception as e:
      pass